In [1]:
import sys
from pathlib import Path

# 루트 디렉토리 설정 (modules 디렉토리가 있는 곳 기준)
project_root = Path.cwd().parent  # 필요에 따라 조정
sys.path.append(str(project_root))

In [3]:
from modules.extract.congress_schedule_extractor import CongressScheduleExtractor
from constants.url_constant import MAIN_CONGRESS_SCHEDULE_URL

schedule_extractor = CongressScheduleExtractor(url=MAIN_CONGRESS_SCHEDULE_URL)
schedule_data = schedule_extractor.extract()

meeting_dates = sorted({item["MEETTING_DATE"] for item in schedule_data})
print("회의 날짜 수:", len(meeting_dates))

2025-06-29 18:58:30.260 | INFO     | modules.base.base_extractor:log_info:12 - [Extractor] 📢 CongressScheduleExtractor: Extracting from https://open.assembly.go.kr/portal/openapi/nekcaiymatialqlxr
2025-06-29 18:58:30.262 | DEBUG    | modules.utils.request_utils:_fetch_single_page:43 - ➡️ 요청 파라미터: {'KEY': None, 'Type': 'json', 'UNIT_CD': '100022', 'pIndex': '1', 'pSize': '100'}
2025-06-29 18:58:30.263 | DEBUG    | modules.utils.request_utils:_fetch_single_page:43 - ➡️ 요청 파라미터: {'KEY': None, 'Type': 'json', 'UNIT_CD': '100022', 'pIndex': '7', 'pSize': '100'}
2025-06-29 18:58:30.262 | DEBUG    | modules.utils.request_utils:_fetch_single_page:43 - ➡️ 요청 파라미터: {'KEY': None, 'Type': 'json', 'UNIT_CD': '100022', 'pIndex': '3', 'pSize': '100'}
2025-06-29 18:58:30.262 | DEBUG    | modules.utils.request_utils:_fetch_single_page:43 - ➡️ 요청 파라미터: {'KEY': None, 'Type': 'json', 'UNIT_CD': '100022', 'pIndex': '4', 'pSize': '100'}
2025-06-29 18:58:30.263 | DEBUG    | modules.utils.request_utils:_fetch

회의 날짜 수: 5


In [4]:
schedule_data[:10]

[{'MEETINGSESSION': '제426회국회(임시회)',
  'CHA': '제3차 ',
  'TITLE': '제426-3차(의사일정)',
  'MEETTING_DATE': '2025-06-27',
  'MEETTING_TIME': '14:00',
  'LINK_URL': 'https://www.assembly.go.kr/portal/na/agenda/agendaSchl.do?menuNo=600015&scheduleDivCd=ASSEM&uniqId=1100018172',
  'UNIT_CD': '100022',
  'UNIT_NM': '제22대',
  'CONTS': '1. 12.29 여객기 참사 진상규명과 피해자 및 유가족의 피해구제를 위한 특별위원회 활동기간 연장의 건<br>2. 국회운영위원장(박찬대) 사임의 건<br>3. 법제사법위원장(정청래) 사임의 건<br>4. 문화체육관광위원장(전재수) 사임의 건<br>5. 국회운영위원장 보궐선거<br>6. 법제사법위원장 보궐선거<br>7. 문화체육관광위원장 보궐선거<br>8. 예산결산특별위원장 선거<br><br>※ 사정에 따라 의사일정이 변경될 수 있음. <br>'},
 {'MEETINGSESSION': '제426회국회(임시회)',
  'CHA': '제2차 ',
  'TITLE': '제426-2차(의사일정)',
  'MEETTING_DATE': '2025-06-26',
  'MEETTING_TIME': '10:00',
  'LINK_URL': 'https://www.assembly.go.kr/portal/na/agenda/agendaSchl.do?menuNo=600015&scheduleDivCd=ASSEM&uniqId=1100018171',
  'UNIT_CD': '100022',
  'UNIT_NM': '제22대',
  'CONTS': '1. 2025년도 제2회 추가경정예산안에 대한 정부의 시정연설<br>2. 찰스 랭글(Charles B. Rangel) 전 미 하원의원 추모 결의안(원안, 최형두의원？김영배의

In [26]:
from collections import Counter

# CONF_DATE만 추출
dates = [item['MEETTING_DATE'] for item in schedule_data if 'MEETTING_DATE' in item]
counter = Counter(dates)

# 중복된 날짜만 필터링
duplicates = {date: count for date, count in counter.items() if count > 1}

# 출력
print(f"🔍 총 날짜 수: {len(dates)}")
print(f"📌 고유 날짜 수: {len(set(dates))}")
print(f"❗ 중복된 날짜 수: {len(duplicates)}개")
for date, count in duplicates.items():
    print(f"- {date}: {count}회")

🔍 총 날짜 수: 500
📌 고유 날짜 수: 5
❗ 중복된 날짜 수: 5개
- 2025-06-27: 100회
- 2025-06-26: 100회
- 2025-06-05: 100회
- 2025-05-01: 100회
- 2025-04-24: 100회


In [ ]:
meeting_dates

['2025-04-24', '2025-05-01', '2025-06-05', '2025-06-26', '2025-06-27']

In [4]:
from modules.extract.congress_schedule_extractor import CongressScheduleExtractor
from modules.extract.pdf_url_extractor import PDFUrlExtractor
from constants.url_constant import MAIN_CONGRESS_SCHEDULE_URL, MAIN_CONGRESS_SPEECH_PDF_URL

if __name__ == "__main__":
    # Step 1: 일정 추출
    schedule_extractor = CongressScheduleExtractor(url=MAIN_CONGRESS_SCHEDULE_URL)
    print("✅ 일정 extractor 시작")
    schedule_data = schedule_extractor.extract()
    print("✅ 일정 데이터 추출 완료")

    # Step 2: 날짜 리스트 생성
    meeting_dates = sorted(set(item["MEETTING_DATE"] for item in schedule_data))

    # Step 3: PDF URL 추출
    pdf_extractor = PDFUrlExtractor(
        url=MAIN_CONGRESS_SPEECH_PDF_URL,
        meeting_dates=meeting_dates,
        unit_cd="22"
    )
    print("✅ PDF extractor 시작")
    pdf_data, fetched_dates = pdf_extractor.extract()
    print(f"✅ PDF 데이터 추출 완료 ({len(pdf_data)}건)")

2025-06-29 18:21:30.232 | INFO     | modules.base.base_extractor:log_info:12 - [Extractor] 📢 CongressScheduleExtractor: Extracting from https://open.assembly.go.kr/portal/openapi/nekcaiymatialqlxr
2025-06-29 18:21:30.233 | INFO     | modules.utils.request_utils:_fetch_single_page:44 - 📡 요청 중... pIndex=1, URL: https://open.assembly.go.kr/portal/openapi/nekcaiymatialqlxr
2025-06-29 18:21:30.233 | INFO     | modules.utils.request_utils:_fetch_single_page:44 - 📡 요청 중... pIndex=6, URL: https://open.assembly.go.kr/portal/openapi/nekcaiymatialqlxr
2025-06-29 18:21:30.233 | INFO     | modules.utils.request_utils:_fetch_single_page:44 - 📡 요청 중... pIndex=3, URL: https://open.assembly.go.kr/portal/openapi/nekcaiymatialqlxr
2025-06-29 18:21:30.233 | INFO     | modules.utils.request_utils:_fetch_single_page:44 - 📡 요청 중... pIndex=4, URL: https://open.assembly.go.kr/portal/openapi/nekcaiymatialqlxr
2025-06-29 18:21:30.233 | INFO     | modules.utils.request_utils:_fetch_single_page:44 - 📡 요청 중... pInd

✅ 일정 extractor 시작


2025-06-29 18:21:30.573 | INFO     | modules.utils.request_utils:_fetch_single_page:59 - ✅ 4 페이지 데이터 추가 (총 5개)
2025-06-29 18:21:30.575 | INFO     | modules.utils.request_utils:_fetch_single_page:44 - 📡 요청 중... pIndex=11, URL: https://open.assembly.go.kr/portal/openapi/nekcaiymatialqlxr
2025-06-29 18:21:30.580 | INFO     | modules.utils.request_utils:_fetch_single_page:59 - ✅ 7 페이지 데이터 추가 (총 5개)
2025-06-29 18:21:30.581 | INFO     | modules.utils.request_utils:_fetch_single_page:44 - 📡 요청 중... pIndex=12, URL: https://open.assembly.go.kr/portal/openapi/nekcaiymatialqlxr
2025-06-29 18:21:30.616 | INFO     | modules.utils.request_utils:_fetch_single_page:59 - ✅ 6 페이지 데이터 추가 (총 5개)
2025-06-29 18:21:30.617 | INFO     | modules.utils.request_utils:_fetch_single_page:59 - ✅ 1 페이지 데이터 추가 (총 5개)
2025-06-29 18:21:30.617 | INFO     | modules.utils.request_utils:_fetch_single_page:59 - ✅ 3 페이지 데이터 추가 (총 5개)
2025-06-29 18:21:30.617 | INFO     | modules.utils.request_utils:_fetch_single_page:59 - ✅ 8 

✅ 일정 데이터 추출 완료
✅ PDF extractor 시작


2025-06-29 18:21:33.462 | INFO     | modules.utils.request_utils:_fetch_single_page:59 - ✅ 7 페이지 데이터 추가 (총 2개)
2025-06-29 18:21:33.463 | INFO     | modules.utils.request_utils:_fetch_single_page:44 - 📡 요청 중... pIndex=11, URL: https://open.assembly.go.kr/portal/openapi/nzbyfwhwaoanttzje
2025-06-29 18:21:33.464 | INFO     | modules.utils.request_utils:_fetch_single_page:59 - ✅ 9 페이지 데이터 추가 (총 2개)
2025-06-29 18:21:33.465 | INFO     | modules.utils.request_utils:_fetch_single_page:44 - 📡 요청 중... pIndex=12, URL: https://open.assembly.go.kr/portal/openapi/nzbyfwhwaoanttzje
2025-06-29 18:21:33.472 | INFO     | modules.utils.request_utils:_fetch_single_page:59 - ✅ 4 페이지 데이터 추가 (총 2개)
2025-06-29 18:21:33.473 | INFO     | modules.utils.request_utils:_fetch_single_page:59 - ✅ 3 페이지 데이터 추가 (총 2개)
2025-06-29 18:21:33.473 | INFO     | modules.utils.request_utils:_fetch_single_page:59 - ✅ 8 페이지 데이터 추가 (총 2개)
2025-06-29 18:21:33.473 | INFO     | modules.utils.request_utils:_fetch_single_page:59 - ✅ 1 

✅ PDF 데이터 추출 완료 (1300건)


In [6]:
pdf_data[:5]

[{'CONFER_NUM': 54598,
  'TITLE': '제22대 제424회 제6차 국회본회의 (2025년 04월 24일)',
  'CLASS_NAME': '국회본회의',
  'DAE_NUM': 22,
  'CONF_DATE': '2025-04-24',
  'SUB_NAME': '1. 2025년도 제1회 추가경정예산안에 대한 정부의 시정연설',
  'VOD_LINK_URL': 'http://w3.assembly.go.kr/main/player.do?menu=1&mc=10&ct1=22&ct2=424&ct3=06&wv=1',
  'CONF_LINK_URL': 'https://record.assembly.go.kr/assembly/viewer/minutes/xml.do?id=54598&type=summary',
  'PDF_LINK_URL': 'https://record.assembly.go.kr/assembly/viewer/minutes/download/pdf.do?id=54598'},
 {'CONFER_NUM': 54598,
  'TITLE': '제22대 제424회 제6차 국회본회의 (2025년 04월 24일)',
  'CLASS_NAME': '국회본회의',
  'DAE_NUM': 22,
  'CONF_DATE': '2025-04-24',
  'SUB_NAME': '2. 산불피해지원대책 특별위원회 구성의 건(의장 제의)(의안번호 2210078)',
  'VOD_LINK_URL': 'http://w3.assembly.go.kr/main/player.do?menu=1&mc=10&ct1=22&ct2=424&ct3=06&wv=1',
  'CONF_LINK_URL': 'https://record.assembly.go.kr/assembly/viewer/minutes/xml.do?id=54598&type=summary',
  'PDF_LINK_URL': 'https://record.assembly.go.kr/assembly/viewer/minutes/download/p